In [1]:
import os
import shutil
import json
import polus.plugins
from polus.plugins import plugins
from pathlib import Path
from pprint import pprint

In [2]:
manifests = [
    'https://raw.githubusercontent.com/bthoughton/polus-plugins/feat/basic/regression/polus-basic-flatfield-correction-plugin/plugin.json',
    'https://raw.githubusercontent.com/bthoughton/polus-plugins/feat/applyflatfield/transforms/images/polus-apply-flatfield-plugin/plugin.json',
    'https://raw.githubusercontent.com/bthoughton/polus-plugins/plugin/smp-training/segmentation/polus-smp-training-plugin/plugin.json',
    'https://raw.githubusercontent.com/bthoughton/polus-plugins/ftl/feat/binarization-threshold/transforms/images/polus-ftl-label-plugin/plugin.json',
    'https://raw.githubusercontent.com/hamshkhawar/polus-plugins/scaled-nyxus/features/nyxus-plugin/plugin.json',
    'https://raw.githubusercontent.com/hamshkhawar/polus-plugins/thresh_plugin/transforms/tabular/tabular-thresholding-plugin/plugin.json',
    'https://raw.githubusercontent.com/bthoughton/polus-plugins/csv-statistics/features/polus-csv-statistics-plugin/plugin.json'
]

In [3]:
for url in manifests:
    polus.plugins.submit_plugin(url, refresh=True)
    
print(plugins.list)

['ApplyFlatfield', 'BasicFlatfieldCorrectionPlugin', 'CsvStatistics', 'DemoSmpTraining_Inference', 'FtlLabel', 'NyxusPlugin', 'TabularThresholdingPlugin']


In [4]:
int_dir = Path('/home/ben/polus-plugins/data/eastman/intensity')
assert int_dir.exists(), "The input directory does not exist"

int_files = [file.name for file in int_dir.iterdir()]
int_files.sort()
print('The number of input images is {}'.format(len(int_files)))
print('\nThe first 5 images:')
pprint(int_files[:5])
print('\nThe last 5 images:')
pprint(int_files[-5:])

file_pattern = 'x0{x}_y0{y}_wx{t}_wy{p}_c{c}.ome.tif'

The number of input images is 36

The first 5 images:
['x01_y01_wx0_wy0_c1.ome.tif',
 'x01_y01_wx0_wy0_c2.ome.tif',
 'x01_y01_wx0_wy1_c1.ome.tif',
 'x01_y01_wx0_wy1_c2.ome.tif',
 'x01_y01_wx0_wy2_c1.ome.tif']

The last 5 images:
['x01_y02_wx2_wy0_c2.ome.tif',
 'x01_y02_wx2_wy1_c1.ome.tif',
 'x01_y02_wx2_wy1_c2.ome.tif',
 'x01_y02_wx2_wy2_c1.ome.tif',
 'x01_y02_wx2_wy2_c2.ome.tif']


In [5]:
out_dir = {}
parent_out_dir = int_dir.parent
workflow = [
    'BasicFlatfieldCorrectionPlugin',
    'ApplyFlatfield',
    'DemoSmpTraining_Inference',
    'FtlLabel',
    'NyxusPlugin',
    'TabularThresholdingPlugin',
    'CsvStatistics'
    ]

if not parent_out_dir.exists():
    os.mkdir(parent_out_dir)

for name in workflow:
    out_dir[name] = parent_out_dir.joinpath(name)
    if out_dir[name].exists():
        continue
        ### CAUTION THIS REMOVES OLD OUT DIRECTORIES
        #shutil.rmtree(out_dir[name])
    elif name :
        os.mkdir(out_dir[name])
    
pprint(out_dir)

{'ApplyFlatfield': PosixPath('/home/ben/polus-plugins/data/eastman/ApplyFlatfield'),
 'BasicFlatfieldCorrectionPlugin': PosixPath('/home/ben/polus-plugins/data/eastman/BasicFlatfieldCorrectionPlugin'),
 'CsvStatistics': PosixPath('/home/ben/polus-plugins/data/eastman/CsvStatistics'),
 'DemoSmpTraining_Inference': PosixPath('/home/ben/polus-plugins/data/eastman/DemoSmpTraining_Inference'),
 'FtlLabel': PosixPath('/home/ben/polus-plugins/data/eastman/FtlLabel'),
 'NyxusPlugin': PosixPath('/home/ben/polus-plugins/data/eastman/NyxusPlugin'),
 'TabularThresholdingPlugin': PosixPath('/home/ben/polus-plugins/data/eastman/TabularThresholdingPlugin')}


In [6]:
flatfield_corr = plugins.BasicFlatfieldCorrectionPlugin
apply_flatfield = plugins.ApplyFlatfield
ftl = plugins.FtlLabel
smp = plugins.DemoSmpTraining_Inference
nyxus = plugins.NyxusPlugin
threshold = plugins.TabularThresholdingPlugin
csv = plugins.CsvStatistics

In [7]:
[inp.name for inp in flatfield_corr.inputs + flatfield_corr.outputs]

['inpDir', 'filePattern', 'darkfield', 'photobleach', 'groupBy', 'outDir']

In [8]:
flatfield_corr.inpDir = int_dir
flatfield_corr.filePattern = file_pattern
flatfield_corr.darkfield = True
flatfield_corr.photobleach = False
flatfield_corr.groupBy = 'xytp'
flatfield_corr.outDir = str(out_dir['BasicFlatfieldCorrectionPlugin'])

for inp in flatfield_corr.inputs + flatfield_corr.outputs:
    print('{} = {}'.format(inp.name, inp.value))

inpDir = /home/ben/polus-plugins/data/eastman/intensity
filePattern = x0{x}_y0{y}_wx{t}_wy{p}_c{c}.ome.tif
darkfield = True
photobleach = False
groupBy = xytp
outDir = /home/ben/polus-plugins/data/eastman/BasicFlatfieldCorrectionPlugin


In [9]:
flatfield_corr.run()

16-Nov-22 10:40:00 - polus.plugins - INFO     - Running container with GPU: --gpus all. BasicFlatfieldCorrectionPlugin version 1.3.1
16-Nov-22 15:40:02 - basicpy.tools.dct_tools - WARNING  - the value of the environment variable BASIC_DCT_BACKEND is not in ["JAX","SCIPY"]
16-Nov-22 15:40:02 - main     - INFO     - Parsing arguments...
16-Nov-22 15:40:02 - main     - INFO     - input_dir = /data/inputs/input0
16-Nov-22 15:40:02 - main     - INFO     - get_darkfield = True
16-Nov-22 15:40:02 - main     - INFO     - get_photobleach = False
16-Nov-22 15:40:02 - main     - INFO     - file_pattern = x0{x}_y0{y}_wx{t}_wy{p}_c{c}.ome.tif
16-Nov-22 15:40:02 - main     - INFO     - output_dir = /data/outputs/output0/images
16-Nov-22 15:40:02 - main     - INFO     - FILE_EXT = .ome.tif
16-Nov-22 15:40:02 - main     - INFO     - Running on 4 processes.
16-Nov-22 15:40:02 - proc 1   - INFO     - x01_y0(1-2)_wx(0-2)_wy(0-2)_c1.ome.tif: Loading images...
16-Nov-22 15:40:02 - proc 2   - INFO     - x01

In [10]:
[inp.name for inp in apply_flatfield.inputs + apply_flatfield.outputs]

['darkPattern',
 'ffDir',
 'brightPattern',
 'imgDir',
 'imgPattern',
 'photoPattern',
 'outDir']

In [11]:
apply_flatfield.imgDir = int_dir
apply_flatfield.imgPattern = file_pattern
apply_flatfield.ffDir = out_dir['BasicFlatfieldCorrectionPlugin']
apply_flatfield.brightPattern = 'x01_y0(1-2)_wx(0-2)_wy(0-2)_c{c}_flatfield.ome.tif'
apply_flatfield.darkPattern = 'x01_y0(1-2)_wx(0-2)_wy(0-2)_c{c}_darkfield.ome.tif'
apply_flatfield.outDir = out_dir['ApplyFlatfield']

for inp in apply_flatfield.inputs + apply_flatfield.outputs:
    print('{} = {}'.format(inp.name, inp.value))

darkPattern = x01_y0(1-2)_wx(0-2)_wy(0-2)_c{c}_darkfield.ome.tif
ffDir = /home/ben/polus-plugins/data/eastman/BasicFlatfieldCorrectionPlugin
brightPattern = x01_y0(1-2)_wx(0-2)_wy(0-2)_c{c}_flatfield.ome.tif
imgDir = /home/ben/polus-plugins/data/eastman/intensity
imgPattern = x0{x}_y0{y}_wx{t}_wy{p}_c{c}.ome.tif
photoPattern = None
outDir = /home/ben/polus-plugins/data/eastman/ApplyFlatfield


In [12]:
apply_flatfield.run()

16-Nov-22 10:40:06 - polus.plugins - INFO     - Running container with GPU: --gpus all. ApplyFlatfield version 1.2.0
16-Nov-22 15:40:06 - main     - INFO     - Parsing arguments...
16-Nov-22 15:40:06 - main     - INFO     - darkPattern = x01_y0(1-2)_wx(0-2)_wy(0-2)_c{c}_darkfield.ome.tif
16-Nov-22 15:40:06 - main     - INFO     - ffDir = /data/inputs/input0/images
16-Nov-22 15:40:06 - main     - INFO     - brightPattern = x01_y0(1-2)_wx(0-2)_wy(0-2)_c{c}_flatfield.ome.tif
16-Nov-22 15:40:06 - main     - INFO     - imgDir = /data/inputs/input1
16-Nov-22 15:40:06 - main     - INFO     - imgPattern = x0{x}_y0{y}_wx{t}_wy{p}_c{c}.ome.tif
16-Nov-22 15:40:06 - main     - INFO     - photoPattern = None
16-Nov-22 15:40:06 - main     - INFO     - outDir = /data/outputs/output0
16-Nov-22 15:40:06 - main     - INFO     - Output file extension = .ome.tif
16-Nov-22 15:40:06 - main     - INFO     - Running with 4 processes.
16-Nov-22 15:40:06 - unshade 2 - INFO     - 100.00% complete
16-Nov-22 15:40

In [13]:
[inp.name for inp in smp.inputs + smp.outputs]

['inferenceMode',
 'imagesInferenceDir',
 'inferencePattern',
 'pretrainedModel',
 'modelName',
 'encoderBase',
 'encoderVariant',
 'encoderWeights',
 'optimizerName',
 'batchSize',
 'imagesTrainDir',
 'labelsTrainDir',
 'trainPattern',
 'imagesValidDir',
 'labelsValidDir',
 'validPattern',
 'device',
 'checkpointFrequency',
 'lossName',
 'maxEpochs',
 'patience',
 'minDelta',
 'outputDir']

In [14]:
smp_file_pattern = 'x0{x}_y0{y}_wx{t}_wy{p}_c1.ome.tif'

smp.imagesInferenceDir = out_dir['ApplyFlatfield']
smp.inferenceMode = 'active'
smp.inferencePattern = smp_file_pattern
smp.pretrainedModel = Path('/home/ben/polus-plugins/data/UnetPlusPlus-MCCLoss-efficientnet-b6-imagenet-Adam/')
smp.batchSize = 10
smp.device = 'gpu'
smp.lossName = 'MCCLoss'
smp.checkpointFrequency = 10
smp.maxEpochs = 10
smp.patience = 10
smp.minDelta = 10.0
smp.outputDir = out_dir['DemoSmpTraining_Inference']

for inp in smp.inputs + smp.outputs:
    print('{} = {}'.format(inp.name, inp.value))

inferenceMode = inferenceMode.active
imagesInferenceDir = /home/ben/polus-plugins/data/eastman/ApplyFlatfield
inferencePattern = x0{x}_y0{y}_wx{t}_wy{p}_c1.ome.tif
pretrainedModel = /home/ben/polus-plugins/data/UnetPlusPlus-MCCLoss-efficientnet-b6-imagenet-Adam
modelName = None
encoderBase = None
encoderVariant = None
encoderWeights = None
optimizerName = None
batchSize = 10
imagesTrainDir = None
labelsTrainDir = None
trainPattern = None
imagesValidDir = None
labelsValidDir = None
validPattern = None
device = gpu
checkpointFrequency = 10
lossName = lossName.MCCLoss
maxEpochs = 10
patience = 10
minDelta = 10.0
outputDir = /home/ben/polus-plugins/data/eastman/DemoSmpTraining_Inference


In [15]:
smp.run()

16-Nov-22 10:40:07 - polus.plugins - INFO     - Running container with GPU: --gpus all. DemoSmpTraining_Inference version 0.5.9
16-Nov-22 15:40:10 - main - INFO - Parsing arguments...
16-Nov-22 15:40:10 - main - INFO - Using device: cuda...
16-Nov-22 15:40:11 - main - INFO - Using input arguments:
16-Nov-22 15:40:11 - main - INFO - 	batchSize = 10
16-Nov-22 15:40:11 - main - INFO - 	checkFreq = 10
16-Nov-22 15:40:11 - main - INFO - 	device = gpu
16-Nov-22 15:40:11 - main - INFO - 	encoderBase = ResNet
16-Nov-22 15:40:11 - main - INFO - 	encoderVariant = resnet34
16-Nov-22 15:40:11 - main - INFO - 	encoderWeights = None
16-Nov-22 15:40:11 - main - INFO - 	imagesInferenceDir = /data/inputs/input0
16-Nov-22 15:40:11 - main - INFO - 	imagesTrainDir = None
16-Nov-22 15:40:11 - main - INFO - 	imagesValidDir = None
16-Nov-22 15:40:11 - main - INFO - 	inferenceMode = active
16-Nov-22 15:40:11 - main - INFO - 	inferencePattern = x0{x}_y0{y}_wx{t}_wy{p}_c1.ome.tif
16-Nov-22 15:40:11 - main - INF

In [16]:
[inp.name for inp in ftl.inputs + ftl.outputs]

['inpDir', 'connectivity', 'binarizationThreshold', 'outDir']

In [17]:
ftl.inpDir = out_dir['DemoSmpTraining_Inference']
ftl.connectivity = 1
ftl.binarizationThreshold = 0.5
ftl.outDir = out_dir['FtlLabel']

for inp in ftl.inputs + ftl.outputs:
    print('{} = {}'.format(inp.name, inp.value))

inpDir = /home/ben/polus-plugins/data/eastman/DemoSmpTraining_Inference
connectivity = 1
binarizationThreshold = 0.5
outDir = /home/ben/polus-plugins/data/eastman/FtlLabel


In [18]:
ftl.run()

16-Nov-22 10:40:40 - polus.plugins - INFO     - Running container with GPU: --gpus all. FtlLabel version 0.3.11
16-Nov-22 15:40:41 - main     - INFO     - Parsing arguments...
16-Nov-22 15:40:41 - main     - INFO     - inpDir = /data/inputs/input0
16-Nov-22 15:40:41 - main     - INFO     - connectivity = 1
16-Nov-22 15:40:41 - main     - INFO     - binarizationThreshold = 0.5
16-Nov-22 15:40:41 - main     - INFO     - outDir = /data/outputs/output0
16-Nov-22 15:40:41 - main     - INFO     - processing 18 images in total...
16-Nov-22 15:40:41 - main     - INFO     - processing 18 small images with cython...
16-Nov-22 15:40:41 - main     - INFO     - processing 0 large images with rust
/ftl-rust/src/main.py:113: DeprecationWarning: in the future the `.dtype` attribute of a given datatype object must be a valid dtype instance. `data_type.dtype` may need to be coerced using `np.dtype(data_type.dtype)`. (Deprecated NumPy 1.20)
  labels = ftl.label_nd(image, connectivity)
16-Nov-22 15:40:42 

In [19]:
# [inp.name for inp in file_renaming.inputs + file_renaming.outputs]

In [20]:
# smp_file_pattern = 'x0{x}_y0{y}_wx{t}_wy{p}_c1.ome.tif'

# file_renaming.inpDir = out_dir['FtlLabel']
# file_renaming.outDir = out_dir['FileRenaming']
# file_renaming.filePattern = 'x0{row:d}_y0{col:d}_wx{well_row:d}_wy{well_col:d}_c1.ome.tif'
# file_renaming.outFilePattern = 'x0{row:d}_y0{col:d}_wx{well_row:d}_wy{well_col:d}_c2.ome.tif'

# for inp in file_renaming.inputs + file_renaming.outputs:
#     print('{} = {}'.format(inp.name, inp.value))

In [21]:
# file_renaming.run()

In [22]:
[inp.name for inp in nyxus.inputs + nyxus.outputs]

['inpDir',
 'segDir',
 'features',
 'filePattern',
 'mapVar',
 'neighborDist',
 'pixelPerMicron',
 'outDir']

In [23]:
nyxus.inpDir = out_dir['ApplyFlatfield']
nyxus.segDir = out_dir['FtlLabel']
nyxus.features = 'BASIC_MORPHOLOGY,ALL_INTENSITY'
nyxus.filePattern = 'x{x+}.*c1.ome.tif' # x([0-9]+).*c2.ome.tif
nyxus.mapVar='c2'
nyxus.neighborDist = 5
nyxus.pixelPerMicron = 0.83612
nyxus.outDir = out_dir['NyxusPlugin']

# 'x0{x}_y0{y}_wx{t}_wy{p}_c1.ome.tif'

In [24]:
nyxus.run()

16-Nov-22 10:40:42 - polus.plugins - INFO     - Running container with GPU: --gpus all. NyxusPlugin version 0.1.2
16-Nov-22 15:40:43 - main     - INFO     - Parsing arguments...
16-Nov-22 15:40:43 - main     - INFO     - inpDir = /data/inputs/input0
16-Nov-22 15:40:43 - main     - INFO     - segDir = /data/inputs/input1
16-Nov-22 15:40:43 - main     - INFO     - filePattern = x{x+}.*c1.ome.tif
16-Nov-22 15:40:43 - main     - INFO     - mapVar = c2
16-Nov-22 15:40:43 - main     - INFO     - features = BASIC_MORPHOLOGY,ALL_INTENSITY
16-Nov-22 15:40:43 - main     - INFO     - neighborDist = 5.0
16-Nov-22 15:40:43 - main     - INFO     - pixelPerMicron = 0.83612
16-Nov-22 15:40:43 - main     - INFO     - outDir = /data/outputs/output0
16-Nov-22 15:40:43 - main     - INFO     - Total number of replicates found: ['01']
16-Nov-22 15:40:43 - main     - INFO     - mapVar: c2
16-Nov-22 15:40:43 - main     - INFO     - filepattern is x01.*c1.ome.tif
16-Nov-22 15:40:43 - main     - INFO     - ['x0

In [25]:
[inp.name for inp in threshold.inputs + threshold.outputs]

['inpDir',
 'metaDir',
 'mappingvariableName',
 'negControl',
 'posControl',
 'variableName',
 'thresholdType',
 'falsePositiverate',
 'numBins',
 'n',
 'outFormat',
 'outDir']

In [26]:
threshold.inpDir = out_dir['NyxusPlugin']
threshold.metaDir = '/home/ben/polus-plugins/data/eastman/meta/'
threshold.outDir = out_dir['TabularThresholdingPlugin']
threshold.mappingvariableName = 'intensity_image'
threshold.negControl = 'virus_negative'
threshold.posControl = 'virus_neutral'
threshold.variableName = 'MEAN'
threshold.thresholdType = 'all'
numBins = 512
threshold.falsePositiverate = 0.1
# threshold.n = '4'
outFormat = 'csv'


In [27]:
threshold.run()

16-Nov-22 10:40:44 - polus.plugins - INFO     - Running container with GPU: --gpus all. TabularThresholdingPlugin version 0.1.0
16-Nov-22 15:40:46 - main     - INFO     - Parsing arguments...
16-Nov-22 15:40:46 - main     - INFO     - inpDir = /data/inputs/input0
16-Nov-22 15:40:46 - main     - INFO     - metaDir = /data/inputs/input1
16-Nov-22 15:40:46 - main     - INFO     - mappingvariableName = intensity_image
16-Nov-22 15:40:46 - main     - INFO     - outDir = /data/outputs/output0
16-Nov-22 15:40:46 - main     - INFO     - negControl = virus_negative
16-Nov-22 15:40:46 - main     - INFO     - posControl = virus_neutral
16-Nov-22 15:40:46 - main     - INFO     - variableName = MEAN
16-Nov-22 15:40:46 - main     - INFO     - thresholdType = all
16-Nov-22 15:40:46 - main     - INFO     - falsePositiverate = 0.1
16-Nov-22 15:40:46 - main     - INFO     - numBins = 512
16-Nov-22 15:40:46 - main     - INFO     - n = 4
16-Nov-22 15:40:46 - main     - INFO     - outFormat = csv
16-Nov-22

In [28]:
# for file in out_dir['NyxusPlugin'].iterdir():
#     with open(file, 'r') as fr:
#         lines = fr.readlines()
#         lines[0] = lines[0].replace('intensity_image', 'file')
#         fr.close()
    
#     with open(file, 'w') as fw:
#         fw.writelines(lines)

In [29]:
[inp.name for inp in csv.inputs + csv.outputs]

['statistics', 'inpDir', 'filePattern', 'groupBy', 'outDir']

In [30]:
csv.inpDir = out_dir['NyxusPlugin']
csv.statistics = 'Mean'
csv.filePattern = 'x0{x}_y0{y}_wx{t}_wy{p}_c2.ome.tif'
csv.groupBy = 'tp'
csv.outDir = out_dir['CsvStatistics']

In [31]:
# csv.run()